March 19, 2024 <br>

# Analysis for Manuscript: *Two-Stage Machine Learning-Based Approach to Predict Points of Departure for Human Non-cancer and Developmental/Reproductive Effects*

Jacob Kvasnicka1, Nicolò Aurisano2, Kerstin von Borries2, En-Hsuan Lu1, Peter Fantke2, Olivier Jolliet2, Fred A. Wright3, Weihsueh A. Chiu*1

1Department of Veterinary Physiology and Pharmacology, Interdisciplinary Faculty of Toxicology, Texas A&M University, College Station, Texas, United States 
2Quantitative Sustainability Assessment, Department of Environmental and Resource Engineering, Technical University of Denmark, Lyngby, Denmark 
3Departments of Statistics and Biological Sciences and Bioinformatics Research Center, North Carolina State University, Raleigh, North Carolina, United States

**Corresponding Author:** Weihsueh A. Chiu, wchiu@tamu.edu <br>
**Notebook Author:** Jacob Kvasnicka <br>

## Setup

In [1]:
import pandas as pd
# Display maximum rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from raw_processing import other_sources
from config_management import UnifiedConfiguration
from data_management import DataManager
from metrics_management import MetricsManager
from results_management import ResultsManager
from results_analysis import ResultsAnalyzer
from plotting.moe import MOE_CATEGORIES

In [2]:
config = UnifiedConfiguration()
data_manager = DataManager(config.data, config.path)
metrics_manager = MetricsManager(config.category_to_dict('metric'))
results_manager = ResultsManager(
    results_file_type=config.data.file_type
)
results_analyzer = ResultsAnalyzer(
    results_manager, 
    data_manager,
    config.plot
)

# Get the modeling instructions for the final models
# These are used to read the corresponding results
instruction_for_model = {
    'general' : {
      'target_effect' : 'general',
      'features_source' : 'opera',
      'ld50_type' : 'predicted',
      'data_condition' : 'missing',
      'select_features' : 'true',
      'estimators' : 'RandomForestRegressor'
    },
    'repro_dev' : {
      'target_effect' : 'repro_dev',
      'features_source' : 'opera',
      'ld50_type' : 'predicted',
      'data_condition' : 'missing',
      'select_features' : 'true',
      'estimators' : 'RandomForestRegressor'  
    }
}

def get_model_key(effect):
    '''
    Helper function to get the model key for the specified effect
    category. 
    
    The model key is a unique identifier for a given model. 
    
    Parameters
    ----------
    effect : str
     Name of effect category: 'general' or 'repro_dev'.
     
    Returns
    -------
    tuple of str
        The corresponding model key.
    '''
    return tuple(instruction_for_model[effect].values())

# Define the percentiles for statistical summary
percentiles = [0.05, 0.5, 0.95]

## Dataset Characterization

Compute the sample sizes at each stage of data filtering. These results are summarized in manuscript **Fig. 1**.

In [3]:
# Get the raw surrogate PODs from Aurisano et al. (2023)
raw_surrogate_pods = other_sources.toxicity_data_and_study_counts_from_excel(
    config.path.raw_surrogate_pods_file, 
    config.raw_data.tox_metric, 
    'dtxsid', 
    config.raw_data.surrogate_tox_data_kwargs
)

for effect in list(raw_surrogate_pods.columns.unique(level=0)):
    N = len(raw_surrogate_pods[effect].dropna())
    print(f'{N} chemicals for {effect}')

5209 chemicals for non-reproductive/developmental effects
4938 chemicals for reproductive/developmental effects


In [4]:
# These are the PODs with > 3 in vivo studies in ToxValDB
surrogate_pods = pd.read_csv(
    config.path.surrogate_pods_file, 
    index_col=0
)

for effect, y in list(surrogate_pods.items()):
    N = len(y.dropna())
    print(f'{N} chemicals for {effect}')

2404 chemicals for general
2999 chemicals for repro_dev


In [5]:
# These are the PODs that passed the QSAR standardization workflow
# These chemicals were used for the POD modeling
for effect in surrogate_pods:    
    _, y = results_analyzer.load_features_and_target(
        **instruction_for_model[effect]
    )
    N = len(y)
    print(f'{N} chemicals for {effect}')    

1791 chemicals for general
2228 chemicals for repro_dev


In [6]:
application_chemicals = data_manager.load_application_chemicals()

print(f'{len(application_chemicals)} application chemicals')

32524 application chemicals


## Performance Evaluation

Generate statistical summaries of model performance scores across replicate models. These results are summarized under the manuscript section, *Performance Evaluation and Benchmarking**.

In [7]:
results_analyzer.describe(
    get_model_key('general'), 
    'performances',
    percentiles=percentiles
).round(2)

metric,root_mean_squared_error,median_absolute_error,r2_score,gsd,gsd_squared
count,150.00,150.00,150.00,150.00,150.00
mean,0.69,0.40,0.48,4.96,23.22
std,0.04,0.02,0.04,0.43,4.02
min,0.61,0.35,0.36,4.11,15.98
5%,0.64,0.37,0.41,4.37,18.01
50%,0.69,0.40,0.48,4.91,22.63
95%,0.76,0.44,0.53,5.70,30.29
max,0.80,0.47,0.57,6.29,36.73


In [8]:
results_analyzer.describe(
    get_model_key('repro_dev'), 
    'performances',
    percentiles=percentiles
).round(2)

metric,root_mean_squared_error,median_absolute_error,r2_score,gsd,gsd_squared
count,150.00,150.00,150.00,150.00,150.00
mean,0.60,0.31,0.49,4.05,15.81
std,0.06,0.02,0.06,0.62,5.08
min,0.52,0.26,0.35,3.29,10.29
5%,0.54,0.28,0.38,3.46,11.39
50%,0.58,0.31,0.49,3.81,13.76
95%,0.72,0.34,0.56,5.26,25.89
max,0.80,0.37,0.58,6.36,37.56


## Feature Importance

In [9]:
important_features = {}  # initialize

for effect in instruction_for_model:
    important_features[effect] = set(
        results_analyzer.get_important_features(
            get_model_key(effect)
        )
    )

In [10]:
# These features are common to both effect categories
important_features['general'].intersection(important_features['repro_dev'])

{'CATMoS_LD50_pred',
 'CombDipolPolariz',
 'ReadyBiodeg_pred_discrete',
 'WS_pred'}

In [11]:
# These features are specific to general effects
important_features['general'].difference(important_features['repro_dev'])

{'MP_pred',
 'P_pred',
 'TopoPolSurfAir',
 'VP_pred',
 'nbN_discrete',
 'ndHBdDon_discrete'}

In [12]:
# These features are specific to repro_dev effects
important_features['repro_dev'].difference(important_features['general'])

{'BCF_pred', 'FUB_pred', 'KM_pred', 'KOA_pred', 'Koc_pred', 'Sp3Sp2HybRatio'}

## Sensitivity Analysis

These results were used to create **Table 1.** *Comparison of performance metrics for QSAR models predicting points of departure*.

In [13]:
keys_without_selection = results_manager.read_model_keys(
    inclusion_string='false'
    )

results_analyzer.summarize_model_performances(
    model_keys=keys_without_selection
)

0.025  \
effect                     model_name                        metric              
General Noncancer          RandomForestRegressor (1,791)     RMSE     0.617446   
                                                             MedAE    0.345249   
                                                             $R^2$    0.427616   
                           GradientBoostingRegressor (1,791) RMSE     0.630407   
                                                             MedAE    0.367059   
                                                             $R^2$    0.402489   
                           Ridge (1,791)                     RMSE     0.668410   
                                                             MedAE    0.389967   
                                                             $R^2$    0.344460   
                           LinearRegression (1,791)          RMSE     0.668690   
                                                             MedAE    0.389858   
                                                             $R^2$    0.343969   
                           XGBRegressor (1,791)              RMSE     0.646318   
                                                             MedAE    0.366062   
                                                             $R^2$    0.325178   
                           SVR (1,791)                       RMSE     0.883227   
                                                             MedAE    0.564089   
                                                             $R^2$   -0.035467   
                           MLPRegressor (1,791)              RMSE     1.498215   
                                                             MedAE    0.569779   
                                                             $R^2$  -44.569156   
                           OPERA w/ Exp. LD50s (1,791)       RMSE     0.621115   
                                                             MedAE    0.358134   
                                                             $R^2$    0.405863   
                           CompTox Features (1,791)          RMSE     0.681206   
                                                             MedAE    0.386630   
                                                             $R^2$    0.301154   
                           RDKit Features (1,789)            RMSE     0.647720   
                                                             MedAE    0.352939   
                                                             $R^2$    0.367660   
                           No Imputation (184)               RMSE     0.441455   
                                                             MedAE    0.272213   
                                                             $R^2$   -0.031052   
Reproductive/Developmental RandomForestRegressor (2,228)     RMSE     0.519718   
                                                             MedAE    0.280855   
                                                             $R^2$    0.379977   
                           GradientBoostingRegressor (2,228) RMSE     0.538304   
                                                             MedAE    0.291164   
                                                             $R^2$    0.364841   
                           Ridge (2,228)                     RMSE     0.572986   
                                                             MedAE    0.333423   
                                                             $R^2$    0.306969   
                           LinearRegression (2,228)          RMSE     0.573032   
                                                             MedAE    0.333364   
                                                             $R^2$    0.306895   
                           XGBRegressor (2,228)              RMSE     0.558227   
                                                             MedAE    0.291223   
                                                       

In [14]:
keys_with_selection = results_manager.read_model_keys(
    inclusion_string='true'
    )

results_analyzer.summarize_model_performances(
    model_keys=keys_with_selection
)

0.025  \
effect                     model_name                            metric             
General Noncancer          RandomForestRegressor (final) (1,791) RMSE    0.631001   
                                                                 MedAE   0.359110   
                                                                 $R^2$   0.402050   
                                                                 RMSE    0.631001   
                                                                 MedAE   0.359110   
                                                                 $R^2$   0.402050   
Reproductive/Developmental RandomForestRegressor (final) (2,228) RMSE    0.523493   
                                                                 MedAE   0.275471   
                                                                 $R^2$   0.363602   
                                                                 RMSE    0.523493   
                                                                 MedAE   0.275471   
                                                                 $R^2$   0.363602   

                                                                            0.050  \
effect                     model_name                            metric             
General Noncancer          RandomForestRegressor (final) (1,791) RMSE    0.640619   
                                                                 MedAE   0.366414   
                                                                 $R^2$   0.408917   
                                                                 RMSE    0.640619   
                                                                 MedAE   0.366414   
                                                                 $R^2$   0.408917   
Reproductive/Developmental RandomForestRegressor (final) (2,228) RMSE    0.539058   
                                                                 MedAE   0.284368   
                                                                 $R^2$   0.381948   
                                                                 RMSE    0.539058   
                                                                 MedAE   0.284368   
                                                                 $R^2$   0.381948   

                                                                            0.500  \
effect                     model_name                            metric             
General Noncancer          RandomForestRegressor (final) (1,791) RMSE    0.691144   
                                                                 MedAE   0.402057   
                                                                 $R^2$   0.479789   
                                                                 RMSE    0.691144   
                                                                 MedAE   0.402057   
                                                                 $R^2$   0.479789   
Reproductive/Developmental RandomForestRegressor (final) (2,228) RMSE    0.580969   
                                                                 MedAE   0.312614   
                                                                 $R^2$   0.494801   
                                                                 RMSE    0.580969   
                                                                 MedAE   0.312614   
                                                                 $R^2$   0.494801   

                                                                            0.950  \
effect                     model_name                            metric             
General Noncancer          RandomForestRegressor (final) (1,791) RMSE    0.755731   
                                                                 MedAE   0.437987   
                                                                 $R^2$   0.532331   
                                                                 RMSE    0.755

## Model Application

### Median points of departure (POD) with prediction intervals
These results correspond to the top panel of **Fig. 4.**

In [15]:
# Get the results for all application chemicals
pod_data = {
    effect : results_analyzer.pod_and_prediction_interval(
        get_model_key(effect),
        inverse_transform=True, 
        normalize=True
    )
    for effect in instruction_for_model
}

# Get the median value and prediction interval
for effect, df in pod_data.items():
    median = df['pod'].quantile()
    # Find the closest index to the computed median value
    abs_diff = (df['pod'] - median).abs()
    idx = abs_diff.idxmin()
    median_data = df.loc[idx]
    print(f"{effect}: {median} ({median_data['lb']}, {median_data['ub']})")

general: 11.218834721209232 (0.818494954674385, 153.77278965864915)
repro_dev: 31.08881947251823 (3.442790465052367, 280.7366822905897)


### Chemicals of concern based on margin of exposure (MOE)
These results correspond to the bottom panel of **Fig. 4**.

In [16]:
# Get the upper bounds of the MOE concern categories
thres_for_concern = {
    k.lower().replace(' ', '_') : bounds[-1] 
    for k, bounds in MOE_CATEGORIES.items()
}

for effect in instruction_for_model:
    print(f'For {effect} effects:')

    results_for_percentile = results_analyzer.moe_and_prediction_intervals(get_model_key(effect))

    # Get the upper bound of exposure uncertainty
    ub_exposure_results = results_for_percentile['95th percentile (mg/kg/day)']
    # Get MOEs for the lower bound of the POD prediction interval
    moes = ub_exposure_results['lb']

    for concern, concern_threshold in thres_for_concern.items():
        where_potential_concern = moes <= concern_threshold
        concern_count = ub_exposure_results.loc[where_potential_concern]['cum_count'][-1]
        print(f'\t{concern_count} chemicals {concern}')

For general effects:
	33277 chemicals low_concern
	2337 chemicals moderate_concern
	497 chemicals high_concern
For repro_dev effects:
	32858 chemicals low_concern
	1458 chemicals moderate_concern
	189 chemicals high_concern


In [17]:
# Get the exposure estimates from the SEEM3 model
exposure_df = data_manager.load_exposure_data().loc[application_chemicals]

exposure_difference = (
    exposure_df['95th percentile (mg/kg/day)'] 
    - exposure_df['5th percentile (mg/kg/day)']
)

typical_uncertainty = exposure_difference.median()

print(f'Typical exposure uncertainty: {round(typical_uncertainty)} log10-units')

Typical exposure uncertainty: 4 log10-units
